In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.keras.models import load_model

model = load_model('./Models/violence_detection.h5')

In [3]:
from collections import deque
import time
import telepot
import cv2
import numpy as np
from tensorflow.keras.models import load_model

def print_results(video, limit=None):
    # Parameters
    trueCount = 0
    imageSaved = 0
    filename = 'savedImage.jpg'
    sendAlert = 0
    location = "Bangalore"
    group_id = 'violence_alerts'

    IMG_SIZE = 128
    FRAME_COUNT = 15
    
    # Load the model
    print("Loading model ...")
    model = load_model('./Models/violence_detection.h5')
    
    # Frame queue for sequence-based prediction
    Q = deque(maxlen=FRAME_COUNT)
    
    # Capture video
    vs = cv2.VideoCapture(video)
    writer = None
    (W, H) = (None, None)
    count = 0
    
    while True:
        # Read the next frame
        (grabbed, frame) = vs.read()
        if not grabbed:
            break

        # Get frame dimensions if not already set
        if W is None or H is None:
            (H, W) = frame.shape[:2]

        # Preprocess the frame
        output = frame.copy()
        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frame = frame[:, :, [2, 1, 0]]  # BGR to RGB
        frame = frame / 255.0

        # Append frame to the queue
        Q.append(frame)

        # When the queue is filled with enough frames for a sequence
        if len(Q) == FRAME_COUNT:
            # Prepare the input as an array for prediction
            input_sequence = np.expand_dims(np.array(Q), axis=0)  # Shape: (1, 15, 128, 128, 3)
            
            # Make a prediction on the sequence
            prediction = model.predict(input_sequence)
            label = (prediction[0] > 0.5).astype(int)

            # Set text color and label based on prediction
            text_color = (0, 0, 255) if label == 1 else (0, 255, 0)
            text = "Violence Detected!" if label == 1 else "Normal"
            
            # Display the prediction on the output frame
            FONT = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(output, text, (35, 50), FONT, 1.25, text_color, 3)
            
            # Count violence detections
            if label == 1:
                trueCount += 1
            
            # Save an image if required
            if trueCount == 50 and imageSaved == 0:
                cv2.imwrite(filename, output)
                imageSaved = 1
            
            # Send alert if required
            if sendAlert == 0 and trueCount == 50:
                bot = telepot.Bot('YOUR_TELEGRAM_BOT_TOKEN')
                timeMoment = time.strftime("%Y-%m-%d %H:%M:%S")
                bot.sendMessage(group_id, f"VIOLENCE ALERT!! \nLOCATION: {location} \nTIME: {timeMoment}")
                bot.sendPhoto(group_id, photo=open(filename, 'rb'))
                sendAlert = 1
            
            # Write output frame to disk if writer initialized
            if writer is None:
                fourcc = cv2.VideoWriter_fourcc(*"MJPG")
                writer = cv2.VideoWriter("recordedVideo.avi", fourcc, 30, (W, H), True)
            writer.write(output)

            # Display output
            cv2.imshow("Frame", output)

            # Clear the queue to start a new sequence
            Q.clear()
        
        # Break on 'q' key press
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
    
    # Cleanup
    print("[INFO] cleaning up...")
    vs.release()
    if writer is not None:
        writer.release()
    cv2.destroyAllWindows()


In [ ]:
print_results('SCVD/SCVD_converted/Test/Normal/t_n002_converted.avi')

Loading model ...


1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
[INFO] cleaning up...


: 

In [5]:
from moviepy.editor import VideoFileClip, concatenate_videoclips

# Read files
vid1 = VideoFileClip("Video/n001_converted.avi")
vid2 = VideoFileClip("Video/n002_converted.avi")
vid3 = VideoFileClip("Video/n003_converted.avi")
vid4 = VideoFileClip("Video/v001_converted.avi")
vid5 = VideoFileClip("Video/v002_converted.avi")

# Concat them
final = concatenate_videoclips([vid1, vid2, vid3,vid4, vid5])

# Write output to the file
final.write_videofile("N.avi",codec='libx264')

ValueError: MoviePy couldn't find the codec associated with the filename. Provide the 'codec' parameter in write_videofile.